In [ ]:
# for testing GC perturbation to Alu and Alu-flanking sequences 

In [43]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import basenji
import pandas as pd 
import numpy as np
import os, psutil, io, gzip, time 
import pysam
import warnings

import matplotlib.pyplot as plt
import seaborn as sns
from pybedtools import BedTool

In [27]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)

In [28]:
hg38_file = '/home/data/genomes/hg38.fa'
hg38_fa = pysam.Fastafile(hg38_file)

In [29]:
supremo_dir='/home/shu_zhang/SuPreMo/'
#os.listdir(supremo_dir)

In [30]:
#input files 
in_file = ''
input_sequences = None
fasta_path = hg38_file
genome = 'hg38'
scores_to_use = ['mse', 'corr']
shift_by = [-1, 0, 1]
out_file = '20240116_test'
out_dir = '/home/shu_zhang/SuPreMo/test_out/'
seq_len = 1048576

revcomp = 'add_revcomp'
augment = True
get_seq = True
get_tracks = False
get_maps = False
get_Akita_scores = True
var_set_size = 0
svlen_limit=700e3


In [31]:
#add option for mutate
#first option is "gc" or "shuffle"
#second and third are flanking len (upstream and downstream of variant)


#make three options
#
#gc options - 
#shuffle options
#

gc_mutate=['gc', 'upstream_len', 'downstream_len']

In [32]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# Adjust inputs from arguments


# Handle paths

import os
from pathlib import Path

# This file path and repo path
repo_path = '/home/shu_zhang/SuPreMo/'

# Output directory and file
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
out_file = os.path.join(out_dir, out_file)


# Data path
chrom_lengths_path = f'{repo_path}/data/chrom_lengths_{genome}'
centromere_coords_path = f'{repo_path}/data/centromere_coords_{genome}'
   

# Adjust shift input: Remove shifts that are outside of allowed range
max_shift = 0.4*seq_len
print(max_shift)
shift_by = [x for x in shift_by if x > -max_shift and x < max_shift]


# Adjust input for taking the reverse complement
if revcomp == 'no_revcomp':
    revcomp_decision = [False]
elif revcomp == 'add_revcomp':
    revcomp_decision = [False, True]
elif revcomp == 'only_revcomp':
    revcomp_decision = [True]

if augment and shift_by == [0] and revcomp == 'no_revcomp':
    shift_by = [-1,0,1]
    revcomp_decision = [False, True]

revcomp_decision_i = revcomp_decision


if input_sequences is not None:
    seq_names = pysam.Fastafile(input_sequences).references

419430.4


In [33]:
# Create dictionaries to save sequences, maps, and disruption score tracks, if specified
if get_seq:
    sequences = {}
    
# # # # # # # # # # # # # # # # # # # # # # # # # # # # #
# Read in (and adjust) data


chrom_lengths = pd.read_table(chrom_lengths_path, header = None, names = ['CHROM', 'chrom_max'])
centromere_coords = pd.read_table(centromere_coords_path, sep = '\t')
fasta_open = pysam.Fastafile(fasta_path)

In [34]:
%cd /home/shu_zhang/SuPreMo/


/home/shu_zhang/SuPreMo


/home/shu_zhang/miniconda3/envs/supremo_akita_env/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [35]:
# Assign necessary values to variables across module

sys.path.insert(0, './scripts/')

# Module 1: reading utilities
import reading_utils
reading_utils.var_set_size = var_set_size


# Module 2: get_seq utilities
import get_seq_utils
get_seq_utils.fasta_open = fasta_open
get_seq_utils.chrom_lengths = chrom_lengths
get_seq_utils.centromere_coords = centromere_coords

get_seq_utils.svlen_limit = svlen_limit
get_seq_utils.seq_length = seq_len
get_seq_utils.half_patch_size = round(seq_len/2)


# Module 2: get_Akita_scores utilities
if get_Akita_scores:
    import get_Akita_scores_utils
    get_Akita_scores_utils.chrom_lengths = chrom_lengths
    get_Akita_scores_utils.centromere_coords = centromere_coords
    
#module 3: seq_mutate utils 
if gc_mutate or motif_mutate:
    import seq_mutate_utils

2024-03-19 21:50:16.800389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2024-03-19 21:50:17.701642: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA
To enable them in other operations,

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 sequence (InputLayer)          [(None, 1048576, 4)  0           []                               
                                ]                                                                 
                                                                                                  
 stochastic_reverse_complement   ((None, 1048576, 4)  0          ['sequence[0][0]']               
 (StochasticReverseComplement)  , ())                                                             
                                                                                                  
 stochastic_shift (StochasticSh  (None, 1048576, 4)  0           ['stochastic_reverse_complement[0
 ift)                                                            ][0]']                     

In [36]:
#use test set variants
colnames = ['CHROM', 'POS', 'END', 'REF', 'ALT', 'SVTYPE', 'SVLEN']

test_sv=pd.read_csv('/home/shu_zhang/SuPreMo/test_data/test_set_edge_cases/test_set_edge_SV.bed', sep='\t', names = colnames)
test_sv_no_bnd=test_sv[test_sv.SVTYPE !='BND']
test_simple=pd.read_csv('/home/shu_zhang/SuPreMo/test_data/test_set_edge_cases/test_set_edge_simple.bed', sep='\t', names = colnames)

In [37]:
variants=test_sv_no_bnd
variants

,CHROM,POS,END,REF,ALT,SVTYPE,SVLEN
0,chr1,244397848,244398348.0,AAACTAAACTAAAATAAACTAAAATAAACTAAAATAAACTAAACTA...,A,DEL,500.0
1,chr1,244397848,244398348.0,A,<DEL>,DEL,500.0
2,chr1,244397848,244398348.0,A,<DUP:TANDEM>,DUP,500.0
3,chr1,244397848,244398348.0,A,<INV>,INV,500.0
4,chr1,244397848,244402848.0,A,<DEL>,DEL,5000.0
5,chr1,244397848,244402848.0,A,<DUP:TANDEM>,DUP,5000.0
6,chr1,244397848,244402848.0,A,<INV>,INV,5000.0
7,chr2,400000,400500.0,CTATGGAACAGGCTGGGGTTACTTGGCTATAGAACAGGCCGGGGCC...,C,DEL,500.0
8,chr2,400000,400500.0,C,<DEL>,DEL,500.0
9,chr2,400000,400500.0,C,<DUP:TANDEM>,DUP,500.0


In [44]:
# # # # # # # # # # # # # # # # # # # # # # # # # # # # #
    # Run: Make Akita predictions and calculate disruption scores

nt = ['A', 'T', 'C', 'G']
    
var_set = 0
var_set_list = []

#print(f'Log file being saved here: {out_file}_log')

while True:
    
    # Read in variants
    #variants = reading_utils.read_input(in_file, var_set)
    if len(variants) == 0:
        break
        
        
    # Index input based on row number and create output with same indexes
    variants['var_index'] = list(range(var_set*var_set_size, var_set*var_set_size + len(variants)))
    variants['var_index'] = variants['var_index'].astype(str)

    
    # If there are multiple alternate alleles, split those into new rows and indexes
    if any([',' in x for x in variants.ALT]):

        variants = (variants
                 .set_index(['CHROM', 'POS', 'REF', 'var_index'])
                 .apply(lambda x: x.str.split(',').explode())
                 .reset_index())

        g = variants.groupby(['var_index'])
        variants.loc[g['var_index'].transform('size').gt(1),
               'var_index'] += '-'+g.cumcount().astype(str)
        
    variant_scores = pd.DataFrame({'var_index':variants.var_index})
    
  
    #usually filtering happens here, removed for testing purposes

   
    # Loop through each row (not index) and get disruption scores 
    for i in range(len(variants))[0:2]:
        print('i', i)

        variant = variants.iloc[i]

        var_index = variant.var_index
        CHR = variant.CHROM
        POS = variant.POS
        REF = variant.REF
        ALT = variant.ALT
            

        if 'SVTYPE' in variants.columns:
            END = variant.END
            SVTYPE = variant.SVTYPE
            SVLEN = variant.SVLEN
        else:
            END = np.nan
            SVTYPE = np.nan
            SVLEN = 0

        for shift in shift_by:
            print('shift', shift)

            # Take reverse complement only with 0 shift
            if shift != 0 & True in revcomp_decision:
                revcomp_decision_i = [False]
            else:
                revcomp_decision_i = revcomp_decision
            #revcomp_decision is list of T/F
            for revcomp in revcomp_decision_i:
                print('revcomp', revcomp)

                try:

                    if revcomp:
                        revcomp_annot = '_revcomp'
                    else:
                        revcomp_annot = ''

                    print('input_sequences', input_sequences)
                    if input_sequences is not None:

                        # Generate sequences_i from sequence input
                        if revcomp_annot == '':
                            sequence_names = [x for x in seq_names if x.startswith(f'{var_index}_{shift}') and 
                                              'revcomp' not in x]
                        elif revcomp_annot == '_revcomp':
                            sequence_names = [x for x in seq_names if x.startswith(f'{var_index}_{shift}{revcomp_annot}')]

                        sequences_i = []
                        print('sequence_names', sequence_names)
                        for sequence_name in sequence_names:
                            sequences_i.append(pysam.Fastafile(input_sequences).fetch(sequence_name, 0, seq_len).upper())

                        sequences_i.append([int(x) for x in sequence_name.split('[')[1].split(']')[0].split('_')])
                        

                    else:

                        # Create sequences_i from variant input
                        #get_sequences_SV (for non-BND) returns:
                        #REF_seq, ALT_seq, [var_rel_pos_REF, var_rel_pos_ALT]
                        #var_rel_pos_REF is variant position relative to the reference sequence 
                        #(relative position of POS, or start of varianat)
                        #var_rel_pos_ALT is variant position relative to reference still? 
                        sequences_i = get_seq_utils.get_sequences_SV(CHR, POS, REF, ALT, END, SVTYPE, shift, revcomp)
                        #print('sequences')
                        #print(len(sequences_i))

                    if get_seq:

                        # Get relative position of variant in sequence
                        var_rel_pos = str(sequences_i[-1]).replace(', ', '_')
                        print('var rel pos', var_rel_pos)

                        for ii in range(len(sequences_i[:-1][:3])): 
                            sequences[f'{var_index}_{shift}{revcomp_annot}_{ii}_{var_rel_pos}'] = sequences_i[:-1][ii]
                        print('sequences i', sequences_i)
                        print(sequences_i)      
                    #shu edit here
                    #insert options post sequence whatever 
                    
                    if gc_mutate:
                        #we want to alter ref_seq in sequences_i
                        #mutate_gc(seq, variant_start, variant_end, posflank, endflank, revcomp, mut_percent):
                        var_rel_pos=sequences_i[-1][0]
                        new_seq=seq_mutate_utils.mutate_gc(sequences_i[0], var_rel_pos, var_rel_pos+SVLEN, 
                                                           0,0, revcomp, 50)
                        
                        #for now, replace the alt seq with the gc mutated seq
                        sequences_i_list=list(sequences_i)
                        sequences_i_list[1]=new_seq
                        sequences_i=tuple(sequences_i_list)
                        
                    
                    if motif_mutate:
                        var_rel_pos=sequences_i[-1][0]
                        new_seq=seq_mutate_utils.shuffle_nucs(sequences_i[0], var_rel_pos, var_rel_pos+SVLEN, 
                                                              posflank, endflank, revcomp)
         
                        
                        #for now, replace the alt seq with the gc mutated seq
                        sequences_i[1]=new_seq 
                            
                    if get_Akita_scores:

                        scores = get_Akita_scores_utils.get_scores(POS, SVTYPE, SVLEN, 
                                                                   sequences_i, scores_to_use, 
                                                                   shift, revcomp, 
                                                                   get_tracks, get_maps)
                        
                        print('akita scores')
                        print(scores)


                        if get_tracks:
                            for track in [x for x in scores.keys() if 'track' in x]:
                                variant_tracks[f'{var_index}_{track}_{shift}{revcomp_annot}'] = scores[track]
                                del scores[track]

                        if get_maps:
                            variant_maps[f'{var_index}_{shift}{revcomp_annot}'] = scores['maps']
                            del scores['maps']

                        for score in scores:
                            variant_scores.loc[variant_scores.var_index == var_index, 
                                               f'{score}_{shift}{revcomp_annot}'] = scores[score]


                    print(str(var_index) + ' (' + str(shift) + f' shift{revcomp_annot})')

                except Exception as e: 

                    print(str(var_index) + ' (' + str(shift) + f' shift{revcomp_annot})' + ': Error:', e)

                    pass
 
 

    # Write sequences to fasta file
    if get_seq:

        if var_set == 0:
            sequences_all = sequences.copy()
        else:
            sequences_all.update(sequences)

    
        
    break
        
    var_set_list.append(var_set)
    var_set += 1

/tmp/ipykernel_498579/1785751975.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variants['var_index'] = list(range(var_set*var_set_size, var_set*var_set_size + len(variants)))
/tmp/ipykernel_498579/1785751975.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  variants['var_index'] = variants['var_index'].astype(str)
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.

In [40]:
ServerApp.iopub_data_rate_limit=1e7

NameError: name 'ServerApp' is not defined

In [ ]:
#THIS CHUNK GOES BEFORE "if get_seq:"

#     # Write scores to data frame
#     if get_Akita_scores:

#         # Take average of augmented sequences
#         if augment:
#             for score in scores:
#                 cols = [x for x in variant_scores.columns if score in x]
#                 variant_scores[f'{score}_mean'] = variant_scores[cols].mean(axis = 1)
#                 variant_scores[f'{score}_median'] = variant_scores[cols].median(axis = 1)
#                 variant_scores.drop(cols, axis = 1, inplace = True)

#         # Convert scores from float to string so you can merge scores for variants with multiple alleles
#         for col in variant_scores.iloc[:,1:].columns:
#             variant_scores[col] = [format(x, '.20f') for x in variant_scores[col]]
            
                
#         # Join scores for alternate alleles, separated by a comma
#         if any(['-' in x for x in variant_scores.var_index]):
#             variant_scores['var_index'] = variant_scores.var_index.str.split('-').str[0]
            
#             variant_scores = (variant_scores
#                               .set_index(['var_index'], drop = False)
#                               .rename(columns = {'var_index':'var_index2'})
#                               .groupby('var_index2')
#                               .transform(','.join)
#                               .reset_index()
#                               .drop_duplicates())
        
#         if var_set == 0:
#             variant_scores.to_csv(f'{out_file}_scores_{var_set}', sep = '\t', index = False)
#         else:
#             variant_scores.to_csv(f'{out_file}_scores_{var_set}', sep = '\t', index = False, header = False)
